In [1]:
import torch, numpy as np
from torch.utils.data import DataLoader, TensorDataset
from model import EmbeddingBlock, FourierVectorize

X = np.load("X_T1.npy")
X_mean = np.load("X_mean.npy")
X_std  = np.load("X_std.npy")
Xn  = (X - X_mean) / (X_std + 1e-6)

D   = 16            # 임베딩 차원
T = X.shape[-1]   # 288
LOW_K  = int(T * .10)  # Low-pass
device = "cuda" if torch.cuda.is_available() else "cpu"

embed = EmbeddingBlock(D).to(device).eval()
fvec  = FourierVectorize(T, k_bins=LOW_K, mode="ri").to(device).eval()

bs= 512
dataset  = TensorDataset(torch.tensor(Xn, dtype=torch.float32))
loader = DataLoader(dataset, batch_size=bs, shuffle=False)

all_psi  = []
with torch.no_grad():
    for (xb,) in loader:
        xb = xb.to(device) 
        H  = embed(xb)
        psi = fvec(H).cpu()
        all_psi.append(psi)
psi = torch.cat(all_psi) 
torch.save(psi, "psi_T1.pt")
print("✅  Saved pre-computed features:", psi.shape)


X_T1.npy, Y_T1.npy 저장: (10911, 4, 288) (10911, 12)
✅  Saved pre-computed features: torch.Size([10911, 3584])
